In [1]:
### Finds pure Nash equilibrium, Pareto optima, and social optima

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Define player class
class Player:
    def __init__(self, playerNum, payoffMatrix, playerPosition):
        self.playerNum      = playerNum
        self.payoffMatrix   = payoffMatrix
        self.playerPosition = playerPosition

In [37]:
Player1 = Player('Player1', [[3,0],[5,1]], 'row')
Player2 = Player('Player2', [[3,5],[0,1]], 'column')
#Player1 = Player('Player1', [[0,0],[5,1]], 'row')
#Player2 = Player('Player2', [[0,0],[8,1]], 'column')

In [4]:
### Find pure strategy Nash equilibrium by iteratively searching whether there's improvement in each quadrant

In [5]:
# Calculate coordinates of each quadrant. These coordinates will be used to calculate distance between cells
# Size is rows/columns in the payoff matrix
size = 2
x = np.arange(0, size, 1)
y = np.arange(0, size, 1)
coordArr = [[(0,0) for j in y] for i in x]
for rowInd, row in enumerate((coordArr)):
    for colInd in range(len(row)):
        coordArr[rowInd][colInd] = ( rowInd, colInd )

In [22]:
# Takes coordinates of a location in the payoff matrix, the payoff matrix, and coordinates of full payoff matrix,
# and determines whether improvement is possible. If improvement is not possible in one direction, return 1. If 
# improvement is not possible in two different directions, then this is a possible pure Nash play.

def CheckNashImprovement(curPos, payoffMatrix, coordArr, playerPosition):
    orthogonal = list()
    for rowInd, row in enumerate((coordArr)):
        for colInd in range(len(row)):
            rowDiff, colDiff = arrDiff[0], arrDiff[1]
            # The row player can only move between rows, and column player can only move between columns.
            if (playerPosition == 'row') & (rowDiff == 1) & (colDiff == 0):
                orthogonal.append(coordArr[rowInd][colInd])
            if (playerPosition == 'column') & (rowDiff == 0) & (colDiff == 1):
                orthogonal.append(coordArr[rowInd][colInd])

    # Current payoff
    curPayoff = payoffMatrix[curPos[0]][curPos[1]]
    # If improvement is not possible by moving to the orthogonal cell, add 1 to list. If all values are 1, then possible play
    # for one player that could constitute a Nash equilibrium
    improvement = list()
    for coord in orthogonal:
        adjPayoff = payoffMatrix[coord[0]][coord[1]]
        if adjPayoff <= curPayoff:
            improvement.append(1)
        else:
            improvement.append(0)
    
    return(improvement)

In [7]:
def CheckParetoImprovement(curPos, payoffMatrix, coordArr):
    
    orthogonal = list()
    for rowInd, row in enumerate((coordArr)):
        for colInd in range(len(row)):
            arrDiff = abs( np.subtract( coordArr[rowInd][colInd], curPos ) )
            rowDiff, colDiff = arrDiff[0], arrDiff[1]
            # The row player can only move between rows, and column player can only move between columns.
            if (playerPosition == 'row') & (rowDiff == 1) & (colDiff == 0):
                orthogonal.append(coordArr[rowInd][colInd])
            if (playerPosition == 'column') & (rowDiff == 0) & (colDiff == 1):
                orthogonal.append(coordArr[rowInd][colInd])

    curPayoffs = [Player1.payoffMatrix[curPos[0]][curPos[1]],Player2.payoffMatrix[curPos[0]][curPos[1]]]

    improvement = list()
    for coord in orthogonal:
        adjPayoff = [Player1.payoffMatrix[coord[0]][coord[1]],Player1.payoffMatrix[coord[0]][coord[1]]]
        if adjPayoff <= curPayoffs:
            improvement.append(1)
        else:
            improvement.append(0)
            
    return(improvement)

In [8]:
# Looks through lists that indicate whether improvement is possible at each position, appending all positions that are
# candidate Nash equilibria.
# For Nash equilibrium, equilType = 'nash'
# For Pareto equilibrium, equilType = 'pareto'
def FindPossibleEquil( payoffs, equilType, playerPosition ):
    
    playerPossibleNash = list()
    for rowInd, row in enumerate((payoffs)):
        for colInd in range(len(row)):
            curPos = (rowInd,colInd)
            if equilType == 'nash':
                improve = CheckNashImprovement(curPos, payoffs, coordArr, playerPosition)
            if equilType == 'pareto':
                improve = CheckParetoImprovement(curPos, payoffs, coordArr, playerPosition)
            if all(x == 1 for x in improve):
                # Append the coordinates of a possible Nash equilibrium for one player.
                playerPossibleNash.append(curPos)
    
    return( playerPossibleNash )

In [9]:
# For each player, loop through each quadrant and see whether the quadrant is optimal (player does not have an incentive 
# to change move). If both players have the same possible Nash, that position is a Nash equilibrium.

In [10]:
# Returns two-player Nash
equilType = 'nash'
list( set(FindPossibleEquil( Player1.payoffMatrix, equilType, Player1.playerPosition )) & 
     set(FindPossibleEquil( Player2.payoffMatrix, equilType, Player2.playerPosition )) )

[(1, 1)]

In [12]:
coordArr

[[(0, 0), (0, 1)], [(1, 0), (1, 1)]]

In [73]:
# Pareto dev
curPos = (0,0)
curPayoff = ( Player1.payoffMatrix[curPos[0]][curPos[1]],
               Player2.payoffMatrix[curPos[0]][curPos[1]] )

orthogonal = list()

for rowInd, row in enumerate((coordArr)):
    for colInd in range(len(row)):
        # Orthogonal cells are either 1 row or one column away. So orthogonal where dist == 1.
        dist = sum( abs( np.subtract( coordArr[rowInd][colInd], curPos ) ) )
        # The row player can only move between rows, and column player can only move between columns.
        if dist == 1:
            orthogonal.append( coordArr[rowInd][colInd] )

improvement = list()

for coord in orthogonal:
    
    orthogPayoff = ( Player1.payoffMatrix[coord[0]][coord[1]],
                     Player2.payoffMatrix[coord[0]][coord[1]] )
    
    print(orthogPayoff)
    # If the orthogonal cell is not strictly greater than the current cell, then it is a candidate for Pareto optimality
    if ( not( \
    all( [ curPayoffs < orthogPayoff for curPayoffs, orthogPayoff in zip(curPayoffs, orthogPayoff) ] ) ) ):
        improvement.append(1)
    else:
        improvement.append(0)
    print(improvement)

(0, 5)
[1]
(5, 0)
[1, 1]


In [71]:
print( curPayoffs, orthogPayoff )
#not( curPayoffs < orthogPayoff )
# [(a > b) for a, b in zip(a,b)]
all( [ curPayoffs < orthogPayoff for curPayoffs, orthogPayoff in zip(curPayoffs, orthogPayoff) ] )

(3, 3) (5, 0)


False

In [67]:
(5,0) < (100,-100)

True

In [50]:
print(curPayoffs)
print(orthogPayoff)

not( curPayoffs < orthogPayoff )

(3, 3)
(0, 5)


True

In [69]:
for i in zip( (1,0),(10,-1) ):
    print(i)

(1, 10)
(0, -1)


In [21]:
allPlayerPayoffs = ( Player1.payoffMatrix[curPos[0]][curPos[0]],
                     Player2.payoffMatrix[curPos[0]][curPos[0]] )

(3, 3)

In [56]:
coordArr[rowInd][colInd]

(1, 1)